In [1]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
 

In [3]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [5]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 

In [6]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [7]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    print("Rows in Each Fold is : ")
    print(fold_size)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


In [8]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [45]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    #print("Folds is : ")
    #print(folds)
    scores = list()
    for fold in folds:
        #print("Iteration")
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        #print("Train Set is : ")
        #print(train_set)
        #print("Test Set is : ")
        #print(test_set)
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [10]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [58]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    print("Number of Instances is : ")
    print(n_instances)
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups: 
        size = float(len(group))
        print("Length of Group is : ")
        print(size)
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        print("Classes is : ")
        print(classes)
        for class_val in classes:
            print("Class Value is : ", class_val)
            p = [row[-1] for row in group].count(class_val) / size
            print("Value of P is : ")
            print(p)
            score += p * p
            print("Scores is : ")
            print(score)
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
        print("Gini is : ")
        print(gini)
    return gini

In [51]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    #print("Class Values is : ", class_values)
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        print("Check 1 ")
        index = randrange(len(dataset[0])-1)
        print("Index is : ", index)
        if index not in features:
            features.append(index)
    print("Features is : ")
    print(features)
    for index in features:
        print("Check 2 : ")
        print("Input Feature is : ", index)
        for row in dataset:
            print("Check 3 : ") 
            print("Row is : ")
            print(row)
            groups = test_split(index, row[index], dataset)
            print("Groups is : ")
            print(groups)
            gini =  gini_index(groups, class_values)
            #print("Gini is : ")
            #print(gini)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}  

In [13]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [14]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [40]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    print("check 4 ")
    root = get_split(train, n_features)
    print("Root Node is : ")
    print(root)
    print("check 5 ")
    split(root, max_depth, min_size, n_features, 1)
    return root

In [16]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
 

In [17]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [18]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [65]:
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        print("Sample is : ")
        print(sample)
        tree = build_tree(sample, max_depth, min_size, n_features)
        print("Built Tree is : ")
        print(tree)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

In [52]:
#//// Executing Code : 

In [20]:
# Test the random forest algorithm
seed(2)
# load and prepare data
filename = 'D:/1.AI/1.Python_Project/Random Forest/Dataset/Family_Status_Encoded.csv'
dataset = load_csv(filename)


In [21]:
print(dataset)

[['0', '1', '2', '3', '4', '5', '9'], ['0', '10', '11', '12', '7', '13', '14'], ['0', '15', '16', '3', '17', '13', '9'], ['0', '19', '16', '12', '4', '5', '14'], ['0', '21', '2', '3', '22', '5', '14'], ['23', '10', '11', '3', '7', '5', '14'], ['23', '1', '16', '12', '4', '13', '9'], ['23', '15', '16', '3', '17', '13', '9'], ['23', '24', '11', '3', '7', '5', '14'], ['23', '25', '2', '3', '4', '13', '14']]


In [22]:
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)

In [23]:
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm

{'9': 0, '14': 1}

In [24]:
print(dataset)

[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 0], [0.0, 10.0, 11.0, 12.0, 7.0, 13.0, 1], [0.0, 15.0, 16.0, 3.0, 17.0, 13.0, 0], [0.0, 19.0, 16.0, 12.0, 4.0, 5.0, 1], [0.0, 21.0, 2.0, 3.0, 22.0, 5.0, 1], [23.0, 10.0, 11.0, 3.0, 7.0, 5.0, 1], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0], [23.0, 15.0, 16.0, 3.0, 17.0, 13.0, 0], [23.0, 24.0, 11.0, 3.0, 7.0, 5.0, 1], [23.0, 25.0, 2.0, 3.0, 4.0, 13.0, 1]]


In [66]:
n_folds = 3
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
print("Number of Features is : ", n_features)
for n_trees in [1 #, 5, 10
               ]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Number of Features is :  2
Rows in Each Fold is : 
3
Sample is : 
[[23.0, 24.0, 11.0, 3.0, 7.0, 5.0, 1], [23.0, 15.0, 16.0, 3.0, 17.0, 13.0, 0], [0.0, 19.0, 16.0, 12.0, 4.0, 5.0, 1], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0]]
check 4 
Check 1 
Index is :  1
Check 1 
Index is :  3
Features is : 
[1, 3]
Check 2 : 
Input Feature is :  1
Check 3 : 
Row is : 
[23.0, 24.0, 11.0, 3.0, 7.0, 5.0, 1]
Groups is : 
([[23.0, 15.0, 16.0, 3.0, 17.0, 13.0, 0], [0.0, 19.0, 16.0, 12.0, 4.0, 5.0, 1], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0], [23.0, 1.0, 16.0, 12.0, 4.0, 13.0, 0]], [[23.0, 24.0, 11.0, 3.0, 7.0, 5.0, 1]])
Number of Instances is : 
6.0
Length of Group is : 
5.0
Classes is : 
[0, 1]
Class Value is :  0
Value of P is : 
0.8
Scores is : 
0.6400000000000001
Class Value is :  1
Value of P is : 
0.2
Scores is : 
0.6800000000000002
Gini is : 
0.26666666666666655
Length of Group is : 
1.0
C